## Build unigram dictionary

In [1]:
import re, sys
import copy
import math
from nltk.corpus import brown
from collections import Counter
from nltk import bigrams, ngrams, trigrams
import numpy as np

data = brown.sents()[:40000]


In [2]:
def preprocess(data):
    processed_data = []
    for sentence in data:
        new_sent = []
        for word in sentence:
            new_word = ''.join(re.findall('[a-z]+',word.lower()))
            if(new_word != ''):
                new_sent.append(new_word)
        if(len(new_sent)!=0):
            processed_data.append(new_sent)
    return processed_data        

def MLE_perplexity(prob,test_sent):
    if(prob==0):
        return ('Log-likelihood: -inf | Perplexity: inf')
    N = len(test_sent.split())
    return ('Log-likelihood: {:0.4f} | Perplexity: {:0.4f}'.format(math.log(prob),pow(prob,-1/N)))


In [13]:
test_sentences = []
outfile = open('./output.txt','w')
try:
    infile = 'input.txt'
    with open(infile,'r') as infile:
        test_sentences = [line.split() for line in infile.readlines()]
except:
    print('specify valid path to input test file as: python Assignment_1_15CS10053 <path>')
    exit(0)

test_sentences = preprocess(test_sentences)
test_sentences = [' '.join(sent) for sent in test_sentences]

unigrams = []
processed_data = preprocess(data)
for sent in processed_data:
    unigrams.extend(sent)
    
unigram_counts=Counter(unigrams)
unigram_total=len(unigrams)
unigram_model = {}

for word in unigram_counts:
    unigram_model[word] = unigram_counts[word]/unigram_total

In [ ]:
import re, sys
import copy
import math
from nltk.corpus import brown
from collections import Counter
from nltk import bigrams, ngrams, trigrams
import numpy as np

data = brown.sents()[:40000]

def preprocess(data):
    processed_data = []
    for sentence in data:
        new_sent = []
        for word in sentence:
            new_word = ''.join(re.findall('[a-z]+',word.lower()))
            if(new_word != ''):
                new_sent.append(new_word)
        if(len(new_sent)!=0):
            processed_data.append(new_sent)
    return processed_data        

def MLE_perplexity(prob,test_sent):
    if(prob==0):
        return ('Log-likelihood: -inf | Perplexity: inf')
    N = len(test_sent.split())
    return ('Log-likelihood: {:0.4f} | Perplexity: {:0.4f}'.format(math.log(prob),pow(prob,-1/N)))

test_sentences = []
outfile = open('./output.txt','w')
try:
    infile = sys.argv[1]
    with open(infile,'r') as infile:
        test_sentences = [line.split() for line in infile.readlines()]
except:
    print('specify valid path to input test file as: python Assignment_1_15CS10053 <path>')
    exit(0)

test_sentences = preprocess(test_sentences)
test_sentences = [' '.join(sent) for sent in test_sentences]

unigrams = []
processed_data = preprocess(data)
for sent in processed_data:
    unigrams.extend(sent)
    
unigram_counts=Counter(unigrams)
unigram_total=len(unigrams)
unigram_model = {}

for word in unigram_counts:
    unigram_model[word] = unigram_counts[word]/unigram_total

specify valid path to input test file as: python Assignment_1_15CS10053 <path>


In [4]:
def get_bigrams(sentences):
    model={}
    for sent in sentences:
         for w1,w2 in ngrams(sent,2, pad_left=True,pad_right=True):
            if w1 not in model:
                model[w1]={}
            if w2 not in model[w1]:
                model[w1][w2]=0
            model[w1][w2]+=1     
    return model

bigram_counts= get_bigrams(processed_data)
bigram_model = copy.deepcopy(bigram_counts)

for w1 in bigram_model:
        tot_count=float(sum(bigram_model[w1].values()))
        for w2 in bigram_model[w1]:
            bigram_model[w1][w2]/=tot_count

def get_trigrams(sentences):
    model={}
    for sent in sentences:
         for w1,w2,w3 in ngrams(sent,3, pad_left=True,pad_right=True):
            if (w1,w2) not in model:
                model[(w1,w2)]={}
            if w3 not in model[(w1,w2)]:
                model[(w1,w2)][w3]=0
            model[(w1,w2)][w3]+=1     
    return model

trigram_counts= get_trigrams(processed_data)
trigram_model = copy.deepcopy(trigram_counts)

for (w1,w2) in trigram_model:
        tot_count=float(sum(trigram_model[(w1,w2)].values()))
        for w3 in trigram_model[(w1,w2)]:
            trigram_model[(w1,w2)][w3]/=tot_count

import matplotlib.pyplot as plt
def zipf_plot(sorted_ngrams,ngram):
    x_axis=[ math.log(i) for i in range(1,len(sorted_ngrams)+1)]
    y_axis= [ math.log(i[1]) for i in sorted_ngrams]
    plt.clf()
    plt.xlabel('Log rank')
    plt.ylabel('Log frequency')
    plt.title('Zipf\'s law for {}'.format(ngram))
    plt.plot(x_axis,y_axis)
    plt.savefig('./zipf_{}.png'.format(ngram))

sorted_unigrams = unigram_counts.most_common()
zipf_plot(sorted_unigrams,'unigrams')

bigram_freq = []
for w1 in bigram_counts:
    for w2 in bigram_counts[w1]:
        bigram_freq.append([(w1,w2),bigram_counts[w1][w2]])

sorted_bigrams = sorted(bigram_freq , key = lambda x: x[1], reverse=True)
zipf_plot(sorted_bigrams,'bigrams')

trigram_freq = []
for (w1,w2) in trigram_counts:
    for w3 in trigram_counts[(w1,w2)]:
        trigram_freq.append([(w1,w2,w3),trigram_counts[(w1,w2)][w3]])

sorted_trigrams = sorted(trigram_freq , key = lambda x: x[1], reverse=True)
zipf_plot(sorted_trigrams,'trigrams')

actual_sorted_bigrams = [bg for bg in sorted_bigrams if (bg[0][0]!=None and bg[0][1]!=None)]
actual_sorted_trigrams = [tg for tg in sorted_trigrams if(not(tg[0][0]==None or tg[0][1]==None or tg[0][2]==None))]

# top ten unigrams, bigrams, trigrams
print('Task 1 : N-gram models without smoothing\n', file=outfile)
print('\nTop ten unigrams:', file=outfile)
print(sorted_unigrams[:10], file=outfile)
print('\nTop ten bigrams:', file=outfile)
print(actual_sorted_bigrams[:10], file=outfile)
print('\nTop ten trigrams:', file=outfile)
print(actual_sorted_trigrams[:10], file=outfile)

In [5]:
sorted_trigrams[0]

[(None, None, 'the'), 5798]

In [26]:
total_possible_trigrams = (len(unigram_counts)+1)**3 # 1 to account for None
n = {0:total_possible_trigrams-len(sorted_trigrams)}
deno = sum([i[1] for i in sorted_trigrams])
rev_trigrams = list(reversed(sorted_trigrams))
for tg in rev_trigrams:
    if(tg[1] not in n):
        n[tg[1]] = 1
    else:
        n[tg[1]] += 1
        

In [28]:
counts = list(n.keys())
for rank in range(0,len(counts)-1):
    n[counts[rank]] = ((rank+1)*n[counts[rank+1]])/(n[counts[rank]]*deno)
n[counts[-1]] = trigram_model[(sorted_trigrams[0][0][0],sorted_trigrams[0][0][1])][sorted_trigrams[0][0][2]]
pGT_trigram = {(None,None):{None:n[0]}}
for tg in rev_trigrams:
    if((tg[0][0],tg[0][1]) in pGT_trigram):
        pGT_trigram[(tg[0][0],tg[0][1])][tg[0][2]] = n[tg[1]]
    else:
         pGT_trigram[(tg[0][0],tg[0][1])] = {tg[0][2]: n[tg[1]]}
            

In [34]:
deno

854309

In [35]:
5798/854309

0.006786771531143884

In [33]:
n[5798]

0.1466363176530096

In [31]:
pGT_trigram[(None,None)]['he']

0.00020367337813367295

In [37]:
(sorted_trigrams[0][0][0],sorted_trigrams[0][0][1])

(None, None)

In [38]:
sorted_trigrams[0][0][2]

'the'

In [39]:
trigram_model[(None,None)]

{'the': 0.1466363176530096,
 'only': 0.0021497218007081437,
 'it': 0.038340920586747596,
 'merger': 2.5290844714213454e-05,
 'however': 0.004248861911987861,
 'implementation': 7.587253414264036e-05,
 'this': 0.026605968639352554,
 'nevertheless': 0.0008851795649974709,
 'failure': 0.00010116337885685381,
 'wards': 2.5290844714213454e-05,
 'these': 0.00834597875569044,
 'regarding': 2.5290844714213454e-05,
 'ask': 0.00020232675771370763,
 'on': 0.006904400606980273,
 'four': 0.00048052604957005563,
 'fulton': 5.058168942842691e-05,
 'mayor': 0.00010116337885685381,
 'his': 0.007486090035407182,
 'they': 0.014921598381385938,
 'attorneys': 5.058168942842691e-05,
 'henry': 0.00015174506828528073,
 'hartsfield': 2.5290844714213454e-05,
 'he': 0.034825493171471926,
 'georgia': 7.587253414264036e-05,
 'robert': 0.0002276176024279211,
 'state': 0.00042994436014162873,
 'in': 0.04008598887202833,
 'despite': 0.0009357612544258978,
 'when': 0.010217501264542236,
 'five': 0.00037936267071320183

In [36]:
trigram_model[(sorted_trigrams[0][0][0],sorted_trigrams[0][0][1])][sorted_trigrams[0][0][2]]

0.1466363176530096

In [18]:
print('\nUnigram test probabilities', file=outfile)
for elem in test_sentences:
    p_val=np.prod([unigram_model[i] for i in elem.split()])
    print('Sequence: {:20} | Unigram prob: {:0.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)

print('\nBigram test probabilities', file=outfile)

for elem in test_sentences:
    p_val=1
    for w1,w2 in bigrams(elem.split(),pad_left=True,pad_right=True):
        if(w1 in bigram_model):
            if(w2 in bigram_model[w1]):
                p_val*=bigram_model[w1][w2]
                continue
        p_val = 0
        break
    print('Sequence: {:20} | Bigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)
    

print('\nTrigram test probabilities', file=outfile)
for elem in test_sentences:
    p_val=1
    for w1,w2,w3 in trigrams(elem.split(),pad_left=True,pad_right=True):
        try:
            p_val*=trigram_model[(w1,w2)][w3]
        except Exception as e:
            p_val=0
            break
    print('Sequence: {:20} | Trigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)

# Laplacian smoothing

k = [1e-4, 1e-3, 1e-2, 0.1, 1]
print('###########################################', file=outfile)
print('Task 2 : Laplacian Smoothing', file=outfile)

for alpha in k:

    print('\nk = {} | Unigram model:'.format(alpha), file=outfile)
    # unigrams
    N = len(sorted_unigrams)     # unseen test unigrams not taken into account
    for elem in test_sentences:
        p_val = 1
        for w1 in elem.split():
            if(w1 in unigram_model):
                p_val *= (unigram_counts[w1]+alpha)/(N*alpha + unigram_total)
            else:
                p_val *= alpha/(alpha*N + len(unigram_model))
        print('Sequence: {:20} | Unigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)

    # bigrams
    print('\nk = {} | Bigram model:'.format(alpha), file=outfile)
    for elem in test_sentences:
        p_val=1
        for w1,w2 in bigrams(elem.split(),pad_left=True,pad_right=True):
            if(w1 in bigram_counts):
                t = sum(bigram_counts[w1].values())
                if(w2 in bigram_counts[w1]):
                    p_val *=  (alpha + bigram_counts[w1][w2])/(alpha*N + t)
                else:
                    p_val *= alpha/(alpha*N + t)
            else:
                p_val *= 1/N    
        print('Sequence: {:20} | Bigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)
    
    # trigrams
    print('\nk = {} | Trigram model:'.format(alpha), file=outfile)
    #N = N**2
    for elem in test_sentences:
        p_val=1
        for w1,w2,w3 in trigrams(elem.split(),pad_left=True,pad_right=True):
            if((w1,w2) in trigram_counts):
                t = sum(trigram_counts[(w1,w2)].values())
                if(w3 in trigram_counts[(w1,w2)]):
                    p_val *= (alpha + trigram_counts[(w1,w2)][w3])/(alpha*N + t)
                else:
                    p_val *= alpha/(alpha*N + t)
            else:
                p_val *= 1/N
        print('Sequence: {:20} | Trigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)

# good turing
print('###########################################', file=outfile)
print('Task 3 : Good Turing Smoothing\n', file=outfile)
total_possible_bigrams = (len(unigram_counts)+1)**2 # 1 to account for None
deno = sum([i[1] for i in sorted_bigrams])
n = {0:total_possible_bigrams-len(sorted_bigrams)}
rev_bigrams = list(reversed(sorted_bigrams))
for bg in rev_bigrams:
    if(bg[1] not in n):
        n[bg[1]] = 1
    else:
        n[bg[1]] += 1
        
counts = list(n.keys())
for rank in range(0,len(counts)-1):
    n[counts[rank]] = ((rank+1)*n[counts[rank+1]])/(n[counts[rank]]*deno)
n[counts[-1]] = bigram_model[sorted_bigrams[0][0][0]][sorted_bigrams[0][0][1]]
pGT_bigram = {None:{None:n[0]}}
for bg in rev_bigrams:
    if(bg[0][0] in pGT_bigram):
        pGT_bigram[bg[0][0]][bg[0][1]] = n[bg[1]]
    else:
         pGT_bigram[bg[0][0]] = {bg[0][1]: n[bg[1]]}
            
print('Bigram model\n', file=outfile)
for elem in test_sentences:
    p_val=1
    for w1,w2 in bigrams(elem.split(),pad_left=True,pad_right=True):
        if(w1 in pGT_bigram):
            if(w2 in pGT_bigram[w1]):
                p_val*=pGT_bigram[w1][w2]
                continue
        p_val *= pGT_bigram[None][None]
    print('Sequence: {:20} | Bigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)
'''
total_possible_trigrams = (len(unigram_counts)+1)**3 # 1 to account for None
n = {0:total_possible_trigrams-len(sorted_trigrams)}
deno = sum([i[1] for i in sorted_trigrams])
rev_trigrams = list(reversed(sorted_trigrams))
for tg in rev_trigrams:
    if(tg[1] not in n):
        n[tg[1]] = 1
    else:
        n[tg[1]] += 1
        
counts = list(n.keys())
for rank in range(0,len(counts)-1):
    n[counts[rank]] = ((rank+1)*n[counts[rank+1]])/(n[counts[rank]]*deno)
n[counts[-1]] = trigram_model[(sorted_trigrams[0][0][0],sorted_trigrams[0][0][1])][sorted_trigrams[0][0][2]]
pGT_trigram = {(None,None):{None:n[0]}}
for tg in rev_trigrams:
    if((tg[0][0],tg[0][1]) in pGT_trigram):
        pGT_trigram[(tg[0][0],tg[0][1])][tg[0][2]] = n[tg[1]]
    else:
         pGT_trigram[(tg[0][0],tg[0][1])] = {tg[0][2]: n[tg[1]]}
            
print('\nTrigram model\n', file=outfile)
for elem in test_sentences:
    p_val=1
    for w1,w2,w3 in trigrams(elem.split(),pad_left=True,pad_right=True):
        if((w1,w2) in pGT_trigram):
            if(w3 in pGT_trigram[(w1,w2)]):
                p_val*=pGT_trigram[(w1,w2)][w3]
                continue
        p_val *= pGT_trigram[(None,None)][None]
    print('Sequence: {:20} | Trigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)

# interpolation

lamda = [0.2,0.5,0.8]
print('###########################################', file=outfile)
print('Task 4 : Interpolation Method', file=outfile)
for l in lamda:
    print('\nLambda : {}'.format(l), file=outfile)
    for elem in test_sentences:
        p_val = 1 
        for w1,w2 in bigrams(elem.split(),pad_left=True,pad_right=True):
            p_bigram = 0        # for unseen bigrams
            p_unigram = 0
            if(w1 in bigram_model):
                if(w2 in bigram_model[w1]):
                    p_bigram = bigram_model[w1][w2]
            if(w2 in unigram_model):
                p_unigram = unigram_model[w2]
            p_val *= (l*p_bigram + (1-l)*p_unigram)
        print('Sequence: {:20} | Bigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)), file=outfile)

outfile.close()
'''

"\ntotal_possible_trigrams = (len(unigram_counts)+1)**3 # 1 to account for None\nn = {0:total_possible_trigrams-len(sorted_trigrams)}\ndeno = sum([i[1] for i in sorted_trigrams])\nrev_trigrams = list(reversed(sorted_trigrams))\nfor tg in rev_trigrams:\n    if(tg[1] not in n):\n        n[tg[1]] = 1\n    else:\n        n[tg[1]] += 1\n        \ncounts = list(n.keys())\nfor rank in range(0,len(counts)-1):\n    n[counts[rank]] = ((rank+1)*n[counts[rank+1]])/(n[counts[rank]]*deno)\nn[counts[-1]] = trigram_model[(sorted_trigrams[0][0][0],sorted_trigrams[0][0][1])][sorted_trigrams[0][0][2]]\npGT_trigram = {(None,None):{None:n[0]}}\nfor tg in rev_trigrams:\n    if((tg[0][0],tg[0][1]) in pGT_trigram):\n        pGT_trigram[(tg[0][0],tg[0][1])][tg[0][2]] = n[tg[1]]\n    else:\n         pGT_trigram[(tg[0][0],tg[0][1])] = {tg[0][2]: n[tg[1]]}\n            \nprint('\nTrigram model\n', file=outfile)\nfor elem in test_sentences:\n    p_val=1\n    for w1,w2,w3 in trigrams(elem.split(),pad_left=True,pad_

In [17]:
outfile = open('dsada.txt','w')

In [19]:
n[0]

2.002379273388184e-10

In [20]:
total_possible_bigrams = (len(unigram_counts)+1)**2 # 1 to account for None
deno = sum([i[1] for i in sorted_bigrams])
n = {0:total_possible_bigrams-len(sorted_bigrams)}

In [22]:
rev_bigrams = list(reversed(sorted_bigrams))
for bg in rev_bigrams:
    if(bg[1] not in n):
        n[bg[1]] = 1
    else:
        n[bg[1]] += 1

In [25]:
n

{0: 1635285525,
 1: 266793,
 2: 39062,
 3: 15202,
 4: 7821,
 5: 4792,
 6: 3194,
 7: 2253,
 8: 1760,
 9: 1247,
 10: 959,
 11: 826,
 12: 629,
 13: 579,
 14: 500,
 15: 409,
 16: 367,
 17: 331,
 18: 272,
 19: 254,
 20: 213,
 21: 201,
 22: 170,
 23: 154,
 24: 145,
 25: 128,
 26: 121,
 27: 92,
 28: 108,
 29: 86,
 30: 81,
 31: 69,
 32: 86,
 33: 75,
 34: 75,
 35: 67,
 36: 68,
 37: 51,
 38: 56,
 39: 40,
 40: 43,
 41: 31,
 42: 43,
 43: 42,
 44: 31,
 45: 28,
 46: 25,
 47: 34,
 48: 37,
 49: 26,
 50: 20,
 51: 21,
 52: 33,
 53: 31,
 54: 25,
 55: 22,
 56: 30,
 57: 19,
 58: 26,
 59: 22,
 60: 14,
 61: 22,
 62: 29,
 63: 17,
 64: 15,
 65: 10,
 66: 8,
 67: 11,
 68: 25,
 69: 13,
 70: 16,
 71: 9,
 72: 14,
 73: 20,
 74: 9,
 75: 9,
 76: 6,
 77: 9,
 78: 14,
 79: 14,
 80: 10,
 81: 5,
 82: 6,
 83: 9,
 84: 4,
 85: 4,
 86: 6,
 87: 5,
 88: 10,
 89: 8,
 90: 10,
 91: 7,
 92: 4,
 93: 9,
 94: 7,
 95: 5,
 96: 6,
 97: 6,
 98: 6,
 99: 1,
 100: 2,
 101: 1,
 102: 2,
 103: 6,
 104: 3,
 105: 1,
 106: 5,
 107: 4,
 108: 5,
 109

In [24]:
deno

814769

In [ ]:

total_possible_bigrams = (len(unigram_counts)+1)**2 # 1 to account for None
deno = sum([i[1] for i in sorted_bigrams])
n = {0:total_possible_bigrams-len(sorted_bigrams)}
rev_bigrams = list(reversed(sorted_bigrams))
for bg in rev_bigrams:
    if(bg[1] not in n):
        n[bg[1]] = 1
    else:
        n[bg[1]] += 1
        
counts = list(n.keys())
for rank in range(0,len(counts)-1):
    n[counts[rank]] = ((rank+1)*n[counts[rank+1]])/(n[counts[rank]]*deno)
n[counts[-1]] = bigram_model[sorted_bigrams[0][0][0]][sorted_bigrams[0][0][1]]
pGT_bigram = {None:{None:n[0]}}
for bg in rev_bigrams:
    if(bg[0][0] in pGT_bigram):
        pGT_bigram[bg[0][0]][bg[0][1]] = n[bg[1]]
    else:
         pGT_bigram[bg[0][0]] = {bg[0][1]: n[bg[1]]}
            
print('Bigram model\n', file=outfile)
for elem in test_sentences:
    p_val=1
    for w1,w2 in bigrams(elem.split(),pad_left=True,pad_right=True):
        if(w1 in pGT_bigram):
            if(w2 in pGT_bigram[w1]):
                p_val*=pGT_bigram[w1][w2]
                continue
        p_val *= pGT_bigram[None][None]

In [11]:
k = [1e-4, 1e-3, 1e-2, 0.1, 1]
print('###########################################')

for alpha in k:
    print(alpha)
    # unigrams
    N = len(sorted_unigrams)     # unseen test unigrams not taken into account
    for elem in test_sentences:
        p_val = 1
        for w1 in elem.split():
            if(w1 in unigram_model):
                p_val *= (unigram_counts[w1]+alpha)/(alpha*N + unigram_total)
            else:
                p_val *= alpha/(alpha*N + len(unigram_model))
        print('Sequence: {:20} | Unigram prob: {:.4e}| {}'.format(elem,p_val,MLE_perplexity(p_val,elem)))


###########################################
0.0001
Sequence: he lived a good life | Unigram prob: 6.3156e-15| Log-likelihood: -32.6957 | Perplexity: 691.6980
Sequence: the man was happy    | Unigram prob: 3.8384e-11| Log-likelihood: -23.9834 | Perplexity: 401.7557
Sequence: the person was good  | Unigram prob: 7.5103e-11| Log-likelihood: -23.3122 | Perplexity: 339.6927
Sequence: the girl was sad     | Unigram prob: 1.5061e-12| Log-likelihood: -27.2215 | Perplexity: 902.6871
Sequence: he won the war       | Unigram prob: 2.0098e-11| Log-likelihood: -24.6304 | Perplexity: 472.2912
0.001
Sequence: he lived a good life | Unigram prob: 6.3143e-15| Log-likelihood: -32.6960 | Perplexity: 691.7278
Sequence: the man was happy    | Unigram prob: 3.8378e-11| Log-likelihood: -23.9835 | Perplexity: 401.7728
Sequence: the person was good  | Unigram prob: 7.5089e-11| Log-likelihood: -23.3123 | Perplexity: 339.7079
Sequence: the girl was sad     | Unigram prob: 1.5059e-12| Log-likelihood: -27.2216 | P